# **Word Games**

To wrap up our projects, we'll create some fun word games that can be used for enjoyment as well as language learning. These projects shouldn't be too technically complex, but they'll let you use your Python skills in a creative and fun way.

## **Unscramble**
For the first game, we will random provide a word where the letters are scrambled, and the user has to guess the unscrambled word.

### Data Preparation
First, let's load the corpus. For this project, we'll use a Latin corpus (but you're welcome to use the English or Uspanteko corpus or your own!

In [ ]:
# Load our data
import util

# REPLACE WITH YOUR CORPUS DIRECTORY
corpus = util.load_raw_text(corpus_directory="../corpora/lat")

#### **Exercise 1**
We need a set of all the unique words in the corpus. Following the approach used in the Spellchecker and Predictive Text projects, do the following:
1. Strip accents
1. Make the text lowercase
1. Tokenize
1. Create and fill a set called `lexicon` with all the unique words.

You may use any function in the `util.py` file.

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">import re

corpus = util.strip_accents(corpus)
corpus = corpus.lower()

def tokenize(text: str):
    return re.findall(word_regex, text)

words = tokenize(corpus)

lexicon = set()
for word in words:
    lexicon.add(word)</code></pre></details>

In [ ]:
word_regex = r"[a-z]+"

# TODO: See above!

print(len(lexicon), "unique words")

Next, let's filter out any words shorter than 4 letters (since those will be too easy) and longer than 8 letters (which will be too difficult).

In [ ]:
filtered_lexicon = set()

for word in lexicon:
    if 4 <= len(word) <= 8:
        filtered_lexicon.add(word)

print(len(filtered_lexicon))

### Scrambling a random word
Let's build a function that chooses a random word and scrambles it.

In [ ]:
import random

def random_scramble(lexicon: set):
    lexicon = list(lexicon)
    
    word = random.choice(lexicon)
    
    # Turn the word into a list of characters 
    word_chars = list(word)
    
    # Shuffle those characters
    random.shuffle(word_chars)
    
    # Re-join the characters into a string
    shuffled = ''.join(word_chars)
    
    return {'shuffled': shuffled, 'original': word}

random_scramble(filtered_lexicon)

### Build an app
Finally, let's use this function to build a loop where the user guesses a word, and the app tells them if it was correct or not.

In [ ]:
import gradio as gr
from typing import Tuple

def scrambler_game(current_word, guess: str):
    """
    If `guess` is the correct word, return 'Correct' and pick a new word. Otherwise, return 'Incorrect'
    Returns (correct_label, scrambled_word, current_word)
    """
    if guess == current_word['original']:
        current_word = random_scramble(filtered_lexicon)
        return ('Correct', current_word['shuffled'], current_word)
    else:
        return ('Incorrect', current_word['shuffled'], current_word)
    

def new_word():
    current_word = random_scramble(filtered_lexicon)
    return ('', current_word['shuffled'], current_word)
                
                
with gr.Blocks(theme=gr.themes.Soft(), title="Latin Unscramble") as unscramble:
    # Start with some initial word
    current_word = gr.State(random_scramble(filtered_lexicon))
    
    gr.Markdown("# Latin Unscramble")
    
    # Notice how we set the initial value based on the State
    scrambled_textbox = gr.Textbox(label="Scrambled", interactive=False, value=current_word.value['shuffled'])
    
    guess_textbox = gr.Textbox(label="Guess")
    guess_button = gr.Button(value="Submit")
    
    new_word_button = gr.Button(value="New Word")
    
    output_textbox = gr.Textbox(label="Result", interactive=False)
    
    guess_button.click(fn=scrambler_game, inputs=[current_word, guess_textbox], outputs=[output_textbox, scrambled_textbox, current_word])
    new_word_button.click(fn=new_word, inputs=[], outputs=[output_textbox, scrambled_textbox, current_word])
    
unscramble.launch()

### **Exercise 2**
Modify the Unscramble game with one or more of the following features:
- Add the ability to get a hint, such as the first letter of the unscrambled word
- Give the user a limited number of guesses to guess each word
- Keep track of how many times the user has gotten the word correct
- Clear the user's guess after they click "Submit"

In [ ]:
# TODO: Your improved word games app!

## **Hangman**

Next, let's make a hangman game using our language. In this game, the computer picks a random word, and the player guesses letters one at a time. The player can only have 6 incorrect letters before they lose the game.

In [ ]:
def create_hangman_clue(word, guessed_letters):
    """
    Given a word and a list of letters, create the correct clue. 
    
    For instance, if the word is 'apple' and the guessed letters are 'a' and 'l', the clue should be 'a _ _ l _'
    """
    clue = ''
    for letter in word:
        if letter in guessed_letters:
            clue += letter + ' '
        else:
            clue += '_ '
    return clue
    

def pick_new_word(lexicon):
    lexicon = list(lexicon)
    
    return {
        'word': random.choice(lexicon),
        'guessed_letters': set(),
        'remaining_chances': 6
    }


def hangman_game(current_state, guess):
    """Update the current state based on the guess."""
    guess = guess.lower()
    
    if guess in current_state['guessed_letters'] or len(guess) > 1:
        # Illegal guess, do nothing
        return (current_state, 'Invalid guess')
    
    current_state['guessed_letters'].add(guess)
    
    if guess not in current_state['word']:
        # Wrong guess
        current_state['remaining_chances'] -= 1
        
        if current_state['remaining_chances'] == 0:
            # No more chances! New word
            current_state = pick_new_word(filtered_lexicon)
            return (current_state, 'You lose!')
        else:
            return (current_state, 'Wrong guess :(')
        
    else:
        # Right guess, check if there's any letters left
        for letter in current_state['word']:
            if letter not in current_state['guessed_letters']:
                # Still letters remaining
                return (current_state, 'Correct guess!')
        
        # If we made it here, there's no letters left.
        current_state = pick_new_word(filtered_lexicon)
        return (current_state, 'You win!')
    

def state_changed(current_state):
    clue = create_hangman_clue(current_state['word'], current_state['guessed_letters'])
    guessed_letters = current_state['guessed_letters']
    remaining_chances = current_state['remaining_chances']
    return (clue, guessed_letters, remaining_chances)


with gr.Blocks(theme=gr.themes.Soft(), title="Latin Hangman") as hangman:
    current_word = gr.State(pick_new_word(filtered_lexicon))
    
    gr.Markdown("# Latin Hangman")
    
    with gr.Row():
        current_word_textbox = gr.Textbox(label="Clue", interactive=False, value=create_hangman_clue(current_word.value['word'], current_word.value['guessed_letters']))
        guessed_letters_textbox = gr.Textbox(label="Guessed letters", interactive=False)
        remaining_chances_textbox = gr.Textbox(label="Remaining chances", interactive=False, value=6)
    
    guess_textbox = gr.Textbox(label="Guess")
    guess_button = gr.Button(value="Submit")
    
    output_textbox = gr.Textbox(label="Result", interactive=False)
    
    guess_button.click(fn=hangman_game, inputs=[current_word, guess_textbox], outputs=[current_word, output_textbox])\
                .then(fn=state_changed, inputs=[current_word], outputs=[current_word_textbox, guessed_letters_textbox, remaining_chances_textbox])
    
hangman.launch()

### **Exercise 3**
Modify the Hangman game with one or more of the following features:
- Add the ability to get a hint, such as a free letter
- When the player loses a game, show the correct word
- Keep track of how many times the player wins and loses

In [ ]:
# TODO: Your improved hangman game

## **Build your own game**
Finally, to really test how much you've learned, pick some word game and build it with a language of your choice. Some ideas include [Wordle](https://www.nytimes.com/games/wordle/index.html), making as many words as possible from a set of letters, or Wheel of Fortune.

In [ ]:
# TODO: Your very own word game

## **Summary**
In this tutorial, we built a few easy and fun games using our language. This included:
- Randomly choosing and shuffling words
- Creating game logic
- More experience with Gradio

Although not as technically complex as some of the other projects, word games are a great way to help with language learning, which is critical for an endangered language.

If you've made it this far, you've hopefully learned a lot about building language technology, even when resources are very limited. We hope you enjoyed BELT and can start using these skills out in the real world!